<a href="https://colab.research.google.com/github/SiddharthKumbharkar/TED-Talk-recommender-using-NLP/blob/main/TED_Talk_Recomendor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing the Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

##Reading and Evaluating the Dataset

In [2]:
ted = pd.read_csv('tedx_dataset.csv')

In [3]:
ted.head()

,idx,main_speaker,title,details,posted,url,num_views
0,8d2005ec35280deb6a438dc87b225f89,Alexandra Auer,The intangible effects of walls,More barriers exist now than at the end of Wor...,Posted Apr 2020,https://www.ted.com/talks/alexandra_auer_the_i...,NaN
1,b3072cd11f40eb57fd259555264476c6,Elizabeth Gilbert,It's OK to feel overwhelmed. Here's what to do...,If you're feeling anxious or fearful during th...,Posted Apr 2020,https://www.ted.com/talks/elizabeth_gilbert_it...,NaN
2,4adc9fee977fa04c357ed4c9b52aa3cc,Butterscotch,"""Accept Who I Am""","Firing off her formidable beatboxing skills, m...",Posted Apr 2020,https://www.ted.com/talks/butterscotch_accept_...,0.0
3,59c641a72b495d522a7932145d6c02b3,Ethan Lisi,What it's really like to have autism,"""Autism is not a disease; it's just another wa...",Posted Apr 2020,https://www.ted.com/talks/ethan_lisi_what_it_s...,NaN
4,d227f2faf6ec185e54436d86463f499b,Daniel Finkel,Can you solve the sea monster riddle?,"According to legend, once every thousand years...",Posted Apr 2020,https://www.ted.com/talks/daniel_finkel_can_yo...,0.0


In [4]:
ted.shape

(4467, 7)

In [5]:
ted.isnull().sum()

,0
idx,0
main_speaker,1
title,0
details,0
posted,0
url,0
num_views,4258


##Data Preprocessing

In [6]:
ted['details'] =  ted['title'] + ' ' + ted['details']

ted = ted[['main_speaker', 'details']]
ted.dropna(inplace = True)


<ipython-input-6-229d58c7b46b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ted.dropna(inplace = True)


In [7]:
print(len(ted['details']))


4466


In [8]:
data = ted.copy()

In [9]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []

for i in ted.index:
    review = re.sub('[^a-zA-Z]', ' ', ted.loc[i, 'details'])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
print(corpus[:5])


['intang effect wall barrier exist end world war ii say design alexandra auer erect one wall unwittingli creat second us versu partit mind compromis collect safeti intrigu result social design project focus two elementari school separ fenc auer encourag us dismantl bias regain perspect thing common', 'ok feel overwhelm next feel anxiou fear coronaviru pandem alon offer hope understand author elizabeth gilbert reflect stay present accept grief come trust strength human spirit resili share genet inherit say virtual convers part ted connect seri host head ted chri anderson head curat helen walter record april', 'accept fire formid beatbox skill musician butterscotch serenad hauntingli beauti backtrack made live stage share piec stori shoutout struggl dark time compassion allow accept even worst day still best say', 'realli like autism autism diseas anoth way think say ethan lisi offer glimps way experi world lisi break mislead stereotyp autism share insight common behavior like stim mask 

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(corpus).toarray()

print(X.shape)

(4466, 5000)


##Training the Model

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(X)

print(cosine_sim.shape)


(4466, 4466)


In [13]:
def recommend_talk(user_input, n=5):

    user_input = re.sub('[^a-zA-Z]', ' ', user_input)
    user_input = user_input.lower()
    user_input = user_input.split()
    user_input = [ps.stem(word) for word in user_input if word not in set(stopwords.words('english'))]
    user_input = ' '.join(user_input)

    user_input_vec = vectorizer.transform([user_input]).toarray()

    user_sim = cosine_similarity(user_input_vec, X).flatten()

    sim_indices = user_sim.argsort()[-n:][::-1]

    return ted.iloc[sim_indices][['main_speaker', 'details']]



##Giving top 5 recommendations

In [14]:
user_input = input("Enter a topic you're interested in: ")
recommended_talks = recommend_talk(user_input, n=5)

print("Recommended TED Talks based on your interest:")
print(recommended_talks)

Enter a topic you're interested in: finance and money
Recommended TED Talks based on your interest:
          main_speaker                                            details
100   Thasunda Duckett  6 ways to improve your relationship with money...
993        Tammy Lally  Let's get honest about our money problems Stru...
918       Adam Carroll  What playing Monopoly with real money taught m...
1861      Don Tapscott  How the blockchain is changing money and busin...
768   Wendy De La Rosa  3 psychological tricks to help you save money ...
